In [116]:
import acquire
import prepare
import scipy.stats as stats

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [104]:
df = acquire.get_new_telco_churn_data()

In [105]:
df.sample(5).T

,526,4393,3476,3268,2585
payment_type_id,2,4,4,2,1
payment_type,Mailed check,Credit card (automatic),Credit card (automatic),Mailed check,Electronic check
customer_id,0013-EXCHZ,2790-XUYMV,6661-EIPZC,5012-YSPJJ,9722-UJOJR
gender,Female,Male,Female,Male,Male
senior_citizen,1,0,0,0,0
partner,Yes,No,Yes,Yes,Yes
dependents,No,Yes,Yes,Yes,Yes
tenure,3,71,48,31,29
phone_service,Yes,Yes,No,Yes,Yes
multiple_lines,No,Yes,No phone service,No,No


In [106]:
df = prep_telco_data(df)

In [107]:
df.sample(5).T

,4118,75,4535,2440,6273
payment_type_id,1,2,2,2,1
payment_type,Electronic check,Mailed check,Mailed check,Mailed check,Electronic check
customer_id,6674-KVJHG,1465-WCZVT,5343-SGUBI,9050-IKDZA,6837-HAEVO
senior_citizen,0,0,0,1,0
partner,0,1,0,0,1
dependents,0,1,0,0,1
tenure,11,3,52,2,69
phone_service,1,1,1,1,1
online_security,0,0,0,0,0
online_backup,0,0,1,0,1


In [108]:
df.dtypes

payment_type_id            int64
payment_type              object
customer_id               object
senior_citizen             int64
partner                    int64
dependents                 int64
tenure                     int64
phone_service              int64
online_security            int64
online_backup              int64
device_protection          int64
tech_support               int64
paperless_billing          int64
monthly_charges          float64
total_charges            float64
churn                      int64
internet_service_type     object
contract_type             object
dtype: object

In [109]:
df = create_dummies(df)

In [117]:
df.sample(5).T

,1637,6756,3673,6471,1152
customer_id,5138-WVKYJ,5736-YEJAX,8229-TNIQA,0813-TAXXS,2974-GGUXS
senior_citizen,0,0,0,0,1
partner,0,0,0,0,1
dependents,0,1,0,0,0
tenure,10,69,4,55,36
phone_service,1,1,1,1,1
online_security,1,1,0,0,0
online_backup,0,1,0,0,1
device_protection,0,1,0,1,0
tech_support,0,1,0,0,0


In [114]:
train, validate, test = telco_split(df)

In [115]:
train.shape, validate.shape, test.shape

((3937, 21), (1688, 21), (1407, 21))

In [ ]:
# create a for loop visual for columns I want to train my model with
for i in df.columns:
    sns.countplot(data=train, x=i, hue='churn')
    plt.show()